In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from re import search

%matplotlib inline

In [25]:
pip install -U selenium

Requirement already up-to-date: selenium in c:\programdata\anaconda3\lib\site-packages (3.141.0)
Note: you may need to restart the kernel to use updated packages.


In [26]:
import sys
print(sys.version)

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


In [28]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from http.cookiejar import CookieJar
import urllib
import urllib.request
import html.parser
import requests
from requests.exceptions import HTTPError
from socket import error as SocketError
from selenium import webdriver
from xml.dom.minidom import parseString
import json        
import xmlrpc.client

base_url="https://www.bigbasket.com/product/get-products"
cat_path="/?slug=dals-pulses"
page="&page="
qry_prm="&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc"

page_vals = ["1","2","3","4","5","6","7","8","9"]

tlcs  = [] #tlc_s
llcn  = [] #llc_n
brand = [] #p_brand
mrp   = [] #mrp
sp    = [] #sp
disc  = [] #dis_val
sku   = [] #sku
desc  = [] #p_desc
purl  = [] #absolute_url

def append (myjson, index, key):
    if key == "tlc_s": 
        tlcs.append(myjson[index])
    elif key == "llc_n":    
        llcn.append(myjson[index])
    elif key == "p_brand":    
        brand.append(myjson[index])
    elif key == "mrp":    
        mrp.append(myjson[index])
    elif key == "sp":    
        sp.append(myjson[index])
    elif key == "dis_val":    
        disc.append(myjson[index])
    elif key == "sku":    
        sku.append(myjson[index])
    elif key == "p_desc":    
        desc.append(myjson[index])
    elif key == "absolute_url":    
        purl.append(myjson[index])

def get_all(myjson, key):
    if type(myjson) == str:
        myjson = json.loads(myjson)
    if type(myjson) is dict:
        for jsonkey in myjson:
            if jsonkey == key:
                append (myjson, jsonkey, key)
            elif type(myjson[jsonkey]) in (list, dict):
                get_all(myjson[jsonkey], key)
    elif type(myjson) is list:
        for item in myjson:
            if item == key:
                append (myjson, item, key)
            elif type(item) in (list, dict):
                get_all(item, key)
    
for page_index in page_vals: 
    req_url = base_url+cat_path+page+page_index+qry_prm
    print (req_url)
    try:
        req=urllib.request.Request(req_url)
        cj = CookieJar()
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
        req.add_header('User-Agent','Mozilla/5.0')
        response = opener.open(req)
        p_json = response.read().decode('utf8', errors='ignore')
        json_object = get_all (p_json, "tlc_s")
        json_object = get_all (p_json, "llc_n")
        json_object = get_all (p_json, "p_brand")
        json_object = get_all (p_json, "mrp")
        json_object = get_all (p_json, "sp")
        json_object = get_all (p_json, "dis_val")
        json_object = get_all (p_json, "sku")
        json_object = get_all (p_json, "p_desc")
        json_object = get_all (p_json, "absolute_url")
        response.close()
    except urllib.request.HTTPError as inst:
        output = format(inst)
        print(output)
    
print ("tlcs", len(tlcs))
print ("llcn", len(llcn))
print ("brand", len(brand))
print ("mrp", len(mrp))
print ("sp", len(sp))
print ("disc", len(disc))
print ("sku", len(sku))
print ("desc", len(desc))
print ("purl", len(purl))


df = pd.DataFrame()
df["product"] = pd.Series(tlcs)
df["category"] = pd.Series(llcn)
df["brand"] = pd.Series(brand)
df["mrp"] = pd.Series(mrp)
df["sp"] = pd.Series(sp)
df["discount"] = pd.Series(disc)
df["sku"] = pd.Series(sku)
df["description"] = pd.Series(desc)
df["url"] = pd.Series(purl)

df.drop (df[df['product'] != 'foodgrains-oil-masala'].index, inplace=True)
df.info()
df.to_csv('BigBasketStaples.csv')

https://www.bigbasket.com/product/get-products/?slug=dals-pulses&page=1&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=dals-pulses&page=2&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=dals-pulses&page=3&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=dals-pulses&page=4&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=dals-pulses&page=5&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=dals-pulses&page=6&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=dals-pulses&page=7&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=dals-pulses&page=8&tab_type=[%22all%22]&sorted_on=popularity&list

### DATA CLEANING

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 5 to 247
Data columns (total 9 columns):
product        173 non-null object
category       173 non-null object
brand          173 non-null object
mrp            173 non-null object
sp             173 non-null object
discount       173 non-null object
sku            173 non-null int64
description    173 non-null object
url            173 non-null object
dtypes: int64(1), object(8)
memory usage: 13.5+ KB


In [30]:
df.columns

Index(['product', 'category', 'brand', 'mrp', 'sp', 'discount', 'sku',
       'description', 'url'],
      dtype='object')

In [31]:
df.head()

,product,category,brand,mrp,sp,discount,sku,description,url
5,foodgrains-oil-masala,Sugar & Jaggery,bb Popular,275.00,216.00,21,30005417,Sugar,/pd/30005417/bb-popular-sugar-5-kg/
6,foodgrains-oil-masala,Sugar & Jaggery,bb Popular,55.00,45.00,18,40019396,Sugar,/pd/40019396/bb-popular-sugar-1-kg/
7,foodgrains-oil-masala,Sugar & Jaggery,bb Popular,110.00,89.00,19,40019397,Sugar,/pd/40019397/bb-popular-sugar-2-kg/
8,foodgrains-oil-masala,Sugar & Jaggery,bb Popular,275.00,216.00,21,30005417,Sugar,/pd/30005417/bb-popular-sugar-5-kg/
31,foodgrains-oil-masala,Atta Whole Wheat,Aashirvaad,490.00,412.00,16,126906,Atta - Whole Wheat,/pd/126906/aashirvaad-atta-whole-wheat-10-kg-p...


In [32]:
#Dropping columns that dont mean much for us here. Dropping sku
df.drop('sku', axis=1, inplace=True)


In [33]:
df.head(3)

,product,category,brand,mrp,sp,discount,description,url
5,foodgrains-oil-masala,Sugar & Jaggery,bb Popular,275.00,216.00,21,Sugar,/pd/30005417/bb-popular-sugar-5-kg/
6,foodgrains-oil-masala,Sugar & Jaggery,bb Popular,55.00,45.00,18,Sugar,/pd/40019396/bb-popular-sugar-1-kg/
7,foodgrains-oil-masala,Sugar & Jaggery,bb Popular,110.00,89.00,19,Sugar,/pd/40019397/bb-popular-sugar-2-kg/


In [34]:
df.columns

Index(['product', 'category', 'brand', 'mrp', 'sp', 'discount', 'description',
       'url'],
      dtype='object')

In [35]:
df.columns=['Product', 'Category', 'Brand', 'MRP', 'ActualPrice', 'Discount', 'ProductDesc','Url']

In [36]:
#From Url column deriving Product Quantity
df=df[~df.Url.str.contains("combo", na=False)]

In [37]:
def getQtforPcs(qt):
    array=qt.split('x')
    array = [int(i) for i in array] 
    return np.prod(array)

In [39]:
def getQuantity(x):
    array = x.split('/')
    qttemp=array[3].split('-')
    size=len(qttemp)
    wt=qttemp[size-1]
    qt=qttemp[size-2]
    if(wt=='kg' or wt=='gm' or wt=='g'):
            if(wt=='g'):
                return qt+'gm'
            else:
                return qt+wt
    elif(wt=='pouch' or wt=='bag'):
        return qttemp[size-3]+qttemp[size-2]
    
df['Quantity'] = df['Url'].apply(lambda x: getQuantity(x))

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 5 to 247
Data columns (total 9 columns):
Product        165 non-null object
Category       165 non-null object
Brand          165 non-null object
MRP            165 non-null object
ActualPrice    165 non-null object
Discount       165 non-null object
ProductDesc    165 non-null object
Url            165 non-null object
Quantity       160 non-null object
dtypes: object(9)
memory usage: 12.9+ KB


In [41]:
print(df.isnull().sum())

Product        0
Category       0
Brand          0
MRP            0
ActualPrice    0
Discount       0
ProductDesc    0
Url            0
Quantity       5
dtype: int64


In [17]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df['Quantity'])

5        5kg
6        1kg
7        2kg
8        5kg
31      None
32      None
33      None
58      None
59      None
60      None
77     500gm
78       1kg
79       1kg
80       1kg
81       2kg
82     500gm
83       1kg
84      None
85      None
86      None
88      None
89       1kg
90       1kg
91     500gm
92     500gm
93      None
94      None
95      None
96      None
97      None
98      None
99      None
100     None
101     None
102     None
103     None
104     None
105     None
106     None
107      1kg
108     None
109     None
110     None
111     None
112      2kg
113      1kg
114      2kg
116     None
117      1kg
118    500gm
119    500gm
120     None
121      1kg
122    500gm
123      2kg
124      1kg
125      1kg
126     None
127     None
128     None
129     None
130     None
131      1kg
132      2kg
133      1kg
134     None
135     None
136     None
137    500gm
138     None
139     None
140     None
141     None
142     None
143     None
144     None
145     None

In [18]:
#Dropping whenever quantity is a combo. As we cant compare combos accross website and provide recommendation.
#Combos can be of any variety, itll be too complex to compare combos.

In [19]:
#df=df[~df.Url.str.contains("combo", na=False)]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 5 to 247
Data columns (total 9 columns):
Product        165 non-null object
Category       165 non-null object
Brand          165 non-null object
MRP            165 non-null object
ActualPrice    165 non-null object
Discount       165 non-null object
ProductDesc    165 non-null object
Url            165 non-null object
Quantity       94 non-null object
dtypes: object(9)
memory usage: 12.9+ KB


In [21]:
str1='5x3'
array=str1.split('x')

In [22]:
array

['5', '3']

In [23]:
import numpy as np
mylist = [ 2, 3, 4, 6] 
result = np.prod(np.array(mylist))  
result

144